In [1]:
#требуемые библиотеки
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings("ignore")

### 1. Объединение данных в DataFrame

In [2]:
#исходные данные
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

df1 = pd.DataFrame({'size': petersburg})
df1['city'] = 'SPb'

df2 = pd.DataFrame({'size': magadan})
df2['city'] = 'Mag'

df = pd.concat([df1, df2], axis=0).reset_index(drop=True)
print(df)

      size city
0   0.0974  SPb
1   0.1352  SPb
2   0.0817  SPb
3   0.1016  SPb
4   0.0968  SPb
5   0.1064  SPb
6   0.1050  SPb
7   0.1033  Mag
8   0.0915  Mag
9   0.0781  Mag
10  0.0685  Mag
11  0.0677  Mag
12  0.0697  Mag
13  0.0764  Mag
14  0.0689  Mag


In [3]:
# Проверка данных на нормальность
#заготовка текстов гипотез
H0 = 'данные распределены нормально'
Ha = 'данные не распределены нормально (мы отвергаем H0)'

#уровень значимости
alpha = 0.05

#проверка данных на нормальность
#тест Шапиро-Уилка
_, p = shapiro(df['size'])
if p > alpha:
    print(f"По тесту Шапиро-Уилка {H0}: {p = :.3f} > {alpha}.")
else:
    print(f"По тесту Шапиро-Уилка {Ha}: {p = :.3f} <= {alpha}.")
    
#Тест Д’Агостино
_, p = normaltest(df['size'])
#normaltest возвращает двустороннюю вероятность для проверки гипотезы
if p > alpha/2:
    print(f"По тесту Д’Агостино {H0}: {p = :.3f} > {alpha/2}.")
else:
    print(f"По тесту Д’Агостино {Ha}: {p = :.3f} <= {alpha/2}.")

По тесту Шапиро-Уилка данные распределены нормально: p = 0.115 > 0.05.
По тесту Д’Агостино данные распределены нормально: p = 0.348 > 0.025.


### 2. Тест на корреляцию

* Данные - размеры мидий - на нормальность проверены.
* Независимая переменная: город происхождения - категориальная.
* Зависимая переименная: размер мидий - количественная.
* Происхождение групп: из разных совокупностей (городов).

Вывод: необходимо использовать независимый t-тест.

In [4]:
H0 = 'Нет значимой разницы между размерами мидий из разных городов.'
Ha = 'Есть значимая разница между размерами мидий из разных городов.'

#уровень значимости
alpha = 0.05

test_results = ttest_ind(df['size'][df['city'] == 'SPb'],
                         df['size'][df['city'] == 'Mag'],
                         equal_var=True)

p = round(test_results[1],3)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.005 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий из разных городов.


### 3. Какова разница в среднем размере мидий.

Для оценки разности размеров мидий:
* оценим разницу в средних значениях размеров diff по городам;
* убедимся визуально, что эта разница заметна;
* проверим, что если из размеров мидий того города, где они крупнее, вычесть diff (назовем полученные разности квазиразмерами), то разница между квазиразмерами и размерами мидий в городе, где вычитание не делалось, незначима.

Проверять можно тем же независимым t-тестом, т.к. вычитание константы только сдвигает среднее, и у нас те же условия для проведения теста, что и ранее, когда мы просто проверяли равенство размеров.

In [5]:
#средний размер мидий:
print('средний размер мидий')
print(df.groupby(by='city')['size'].mean())
print()

#оценка раздницы размеров
avg_spb = round(df['size'][df['city'] == 'SPb'].mean() * 100, 1)
avg_mag = round(df['size'][df['city'] == 'Mag'].mean() * 100, 1)
diff_size = round(abs(avg_spb - avg_mag), 1)

print(f'В Санкт-Петербурге мидии в среднем {avg_spb} см, в Магадане - {avg_mag} см. Оценка разности - {diff_size} см')
if avg_spb > avg_mag:
    print('Питерская мидия крупнее')
else:
    print('Питерская мидия мельче')

средний размер мидий
city
Mag    0.078012
SPb    0.103443
Name: size, dtype: float64

В Санкт-Петербурге мидии в среднем 10.3 см, в Магадане - 7.8 см. Оценка разности - 2.5 см
Питерская мидия крупнее


In [6]:
#гипотезы:
H0 = f'Разница в размерах мидии из Петербурга и мидий из Магадана равна {diff_size} см'
Ha = f'Разница в размерах мидии из Петербурга и мидии из Магадана не равна {diff_size} см'

print(f'Для проверки гипотезы H0 надо вычесть из размеров мидий из Петербурга {diff_size} см')
print('и проверить, что тогда разница в размерах незначима.')

#уровень значимости
alpha = 0.05

test_results = ttest_ind(df['size'][df['city'] == 'SPb'] - diff_size / 100,
                         df['size'][df['city'] == 'Mag'],
                         equal_var=True)

p = round(test_results[1],3)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")


Для проверки гипотезы H0 надо вычесть из размеров мидий из Петербурга 2.5 см
и проверить, что тогда разница в размерах незначима.
0.955 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Разница в размерах мидии из Петербурга и мидий из Магадана равна 2.5 см


In [7]:
print('но гипотеза подобного типа может выполниться для широкого круга значений')

diff_variants = [round(diff_size + k * 0.2, 1) for k in range(-10, 11, 1)]
for diff_variant in diff_variants:
    #уровень значимости
    alpha = 0.05

    H0 = f'Мидия из Петербурга в среднем на {diff_variant} см больше, чем в Магадане'
    Ha = f'Разница в размерах мидии из Петербурга и мидии из Магадана не равна {diff_variant} см'

    test_results = ttest_ind(df['size'][df['city'] == 'SPb'] - diff_variant / 100,
                             df['size'][df['city'] == 'Mag'],
                             equal_var=True)

    p = round(test_results[1],3)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")
        
print('но про доверительные интервалы мы узнаем на следующих занятиях')

но гипотеза подобного типа может выполниться для широкого круга значений
0.018 <= 0.05. Мы отвергаем нулевую гипотезу. Разница в размерах мидии из Петербурга и мидии из Магадана не равна 0.5 см
0.029 <= 0.05. Мы отвергаем нулевую гипотезу. Разница в размерах мидии из Петербурга и мидии из Магадана не равна 0.7 см
0.048 <= 0.05. Мы отвергаем нулевую гипотезу. Разница в размерах мидии из Петербурга и мидии из Магадана не равна 0.9 см
0.077 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Мидия из Петербурга в среднем на 1.1 см больше, чем в Магадане
0.123 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Мидия из Петербурга в среднем на 1.3 см больше, чем в Магадане
0.189 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Мидия из Петербурга в среднем на 1.5 см больше, чем в Магадане
0.283 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Мидия из Петербурга в среднем на 1.7 см больше, чем в Магадане
0.408 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Мидия из Петербурга в среднем на 1.9 